In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly import graph_objects as go
from plotly import express as px
import imageio
import json

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [65]:
import yaml
from types import SimpleNamespace
from collections.abc import Iterable

In [264]:
def plot_poses(poses, k=0.3, text=None, marker_color=None, fontsize=None, only_points=False):
    '''
        Draw camera position with plotly
        :param poses - np.ndarray [N, 4, 4]
            cam2world
        :param k - float or np.ndarray.
            length of the axis. default = 0.3
    '''
    # distance to point from (0, 0, 0)
    r = poses if only_points else poses[..., -1][:, [0, 1, 2]]

    # plot all origins
    fig = go.Figure(
        go.Scatter3d(
            x=r[:, 0], 
            y=r[:, 1], 
            z=r[:, 2], 
            marker=dict(size=3, color=marker_color),
            text=text,
            mode="markers+text",
            textfont=dict(
            size=fontsize,
            ),
        )
    )

    def plot_segment(begin, delta, k, color):
        '''
            Function to plot segment in plotly
        '''
        end = begin + k * delta
        return go.Scatter3d(
                x=[begin[0], end[0]], 
                y=[begin[1], end[1]], 
                z=[begin[2], end[2]], 
                line=dict(
                    color=color,
                    width=2
                ),
                mode="lines"
            )

    if isinstance(k, Iterable):
        kx = k[0]
        ky = k[1]
        kz = k[2]
    else:
        kx = ky = kz = k

    if not only_points:
        for pos in poses:
            xax = pos[:3, 0]
            yax = pos[:3, 1]
            zax = pos[:3, 2]
            point_begin = pos[:3, 3]
            fig.add_trace(plot_segment(point_begin, xax, kx, "red"))
            fig.add_trace(plot_segment(point_begin, yax, ky, "green"))
            fig.add_trace(plot_segment(point_begin, zax, kz, "blue"))

    # fig.update_traces(marker_size=3)
    return fig

In [4]:
def qvec2rotmat(qvec):
    return np.array([
        [
            1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
            2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
            2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]
        ], [
            2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
            1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
            2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]
        ], [
            2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
            2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
            1 - 2 * qvec[1]**2 - 2 * qvec[2]**2
        ]
    ])

def rotmat(a, b):
	a, b = a / np.linalg.norm(a), b / np.linalg.norm(b)
	v = np.cross(a, b)
	c = np.dot(a, b)
	# handle exception for the opposite direction input
	if c < -1 + 1e-10:
		return rotmat(a + np.random.uniform(-1e-2, 1e-2, 3), b)
	s = np.linalg.norm(v)
	kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
	return np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2 + 1e-10))

def closest_point_2_lines(oa, da, ob, db): # returns point closest to both rays of form o+t*d, and a weight factor that goes to 0 if the lines are parallel
    da = da / np.linalg.norm(da)
    db = db / np.linalg.norm(db)
    c = np.cross(da, db)
    denom = np.linalg.norm(c)**2
    t = ob - oa
    ta = np.linalg.det([t, db, c]) / (denom + 1e-10)
    tb = np.linalg.det([t, da, c]) / (denom + 1e-10)
    if ta > 0:
        ta = 0
    if tb > 0:
        tb = 0
    return (oa+ta*da+ob+tb*db) * 0.5, denom

In [51]:
poses_colmap = []
poses_colmap_names = []
with open("/home/sevashasla/coding/datasets/tools_all/colmap_text/images.txt", "r") as f:
    i = 0

    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])

    frames = []

    up = np.zeros(3)
    for line in f:
        line = line.strip()

        if line[0] == "#":
            continue

        i = i + 1

        if i % 2 == 1:
            elems = line.split(" ") # 1-4 is quat, 5-7 is trans, 9ff is filename (9, if filename contains no spaces)

            name = '_'.join(elems[9:])
            
            poses_colmap_names.append(name.split("_")[0] + "/" + "_".join(name.split("_")[1:]))

            image_id = int(elems[0])
            qvec = np.array(tuple(map(float, elems[1:5])))
            tvec = np.array(tuple(map(float, elems[5:8])))
            R = qvec2rotmat(-qvec)
            t = tvec.reshape([3, 1])
            m = np.concatenate([np.concatenate([R, t], 1), bottom], 0)
            c2w = np.linalg.inv(m)
            poses_colmap.append(c2w)
poses_colmap = np.array(poses_colmap)
r_colmap = poses_colmap[..., -1][:, [0, 1, 2]]

In [52]:
# poses_colmap = []
# poses_colmap_names = []
# with open("/home/sevashasla/coding/datasets/tools_all/transforms_test.json") as f:
#     data = json.load(f)
#     poses_colmap.extend([el['transform_matrix'] for el in data['frames']])
#     # poses_colmap_names.extend([el['transform_matrix'] for el in data['frames']])

# with open("/home/sevashasla/coding/datasets/tools_all/transforms_train.json") as f:
#     data = json.load(f)
#     poses_colmap.extend([el['transform_matrix'] for el in data['frames']])

# with open("/home/sevashasla/coding/datasets/tools_all/transforms_val.json") as f:
#     data = json.load(f)
#     poses_colmap.extend([el['transform_matrix'] for el in data['frames']])
# poses_colmap = np.array(poses_colmap)
# r_colmap = poses_colmap[..., -1][:, [0, 1, 2]]

In [314]:
poses_my = []
poses_my_names = []
with open("/home/sevashasla/coding/datasets/tools/transforms/transforms_train.json") as f:
    data = json.load(f)
    poses_my.extend([el['transform_matrix'] for el in data['frames']])
    poses_my_names.extend([el['file_path'] for el in data['frames']])

with open("/home/sevashasla/coding/datasets/tools/transforms/transforms_test.json") as f:
    data = json.load(f)
    poses_my.extend([el['transform_matrix'] for el in data['frames']])
    poses_my_names.extend([el['file_path'] for el in data['frames']])

with open("/home/sevashasla/coding/datasets/tools/transforms/transforms_val.json") as f:
    data = json.load(f)
    poses_my.extend([el['transform_matrix'] for el in data['frames']])
    poses_my_names.extend([el['file_path'] for el in data['frames']])
poses_my = np.array(poses_my)
# poses_my = poses_my @ np.linalg.inv(np.array([
#             [1, 0, 0, -0.03284863],
#             [0, 1, 0, -0.08],
#             [0, 0, 1, -0.08414625],
#             [0, 0, 0, 1],
#         ]))
r_my = poses_my[..., -1][:, [0, 1, 2]]

In [ ]:
plot_poses(poses_colmap, k=np.array([0.6, 0.6, 0.3]), text=poses_colmap_names, fontsize=8)

In [ ]:
plot_poses(poses_my, k=[0.03, 0.03, 0.03], text=poses_my_names, fontsize=8)

### Теперь посмотрим на преобразованные

In [315]:
poses_colmap = []
poses_colmap_names = []
change_name = lambda x: x[4:].split("_")[0] + "/" + "_".join(x[4:].split("_")[1:])
with open("/home/sevashasla/coding/datasets/tools_all/transforms.json") as f:
    data = json.load(f)
    poses_colmap.extend([el['transform_matrix'] for el in data['frames']])
    poses_colmap_names.extend([change_name(el['file_path']) for el in data['frames']])
poses_colmap = np.array(poses_colmap)

bad_idx = poses_colmap_names.index("test/r_0.png")

poses_colmap = np.delete(poses_colmap, bad_idx, axis=0)
del poses_colmap_names[bad_idx]

r_colmap = poses_colmap[..., -1][:, [0, 1, 2]]

In [316]:
poses_my = []
poses_my_names = []
with open("/home/sevashasla/coding/datasets/tools/transforms.json") as f:
    data = json.load(f)
    poses_my.extend([el['transform_matrix'] for el in data['frames']])
    poses_my_names.extend([el['file_path'] for el in data['frames']])

poses_my = np.array(poses_my)
# poses_my = poses_my @ np.linalg.inv(np.array([
#             [1, 0, 0, -0.03284863],
#             [0, 1, 0, -0.08],
#             [0, 0, 1, -0.08414625],
#             [0, 0, 0, 1],
#         ]))
r_my = poses_my[..., -1][:, [0, 1, 2]]

### Попробуем понять, как отличаются позы

In [317]:
import open3d as o3d
import copy
import scipy

In [318]:
pcd_colmap = o3d.geometry.PointCloud()
r_colmap_aboba = r_colmap.copy()
r_colmap_aboba -= r_colmap_aboba.mean(axis=0)

pcd_colmap.points = o3d.utility.Vector3dVector(r_colmap_aboba)
o3d.io.write_point_cloud("./colmap.pcd", pcd_colmap)

r_my_aboba = r_my.copy()
r_my_aboba -= r_my_aboba.mean(axis=0)

pcd_my = o3d.geometry.PointCloud()
pcd_my.points = o3d.utility.Vector3dVector(r_my_aboba)
o3d.io.write_point_cloud("./my.pcd", pcd_my)

True

**Потому нужно сделать замену: x <-> -x, swap(x, y)?**

In [327]:
np.linalg.norm((poses_colmap_aboba[:, :3, :] - poses_my_aboba)[:, :3, 3], axis=1)

array([1.37958289, 1.30101477, 2.16880342, 2.65705953, 4.04035189,
       2.86793699, 2.37582629, 1.95229133, 2.33913077, 1.03939256,
       0.01872276, 0.03620324, 0.04830747, 0.16820813, 1.98778858,
       0.6506133 , 1.60805649, 2.43325717, 2.19837692, 3.09563011,
       2.90501257, 1.29926588, 0.32117627, 2.7783503 , 2.17253213,
       1.80031802, 2.25908572, 1.89528801, 2.21940685, 1.7008617 ,
       2.49507553, 2.14150215, 1.22411248, 1.34870386, 0.88810614,
       2.86555958, 1.35621393, 3.96107333, 2.40195065, 3.34312524])

In [356]:
r_colmap = poses_colmap_aboba[:, :3, 3][np.argsort(poses_colmap_names)]
r_my = poses_my_aboba[:, :3, 3][np.argsort(poses_my_names)]
np.abs(r_colmap - r_my).mean(axis=0)

array([0.0709416 , 0.07101364, 0.01255503])

In [357]:
poses_colmap_aboba = poses_colmap.copy()
poses_colmap_aboba[:, :3, 3] -= poses_colmap_aboba[:, :3, 3].mean(axis=0)

poses_my_aboba = poses_my.copy()
poses_my_aboba[:, :3, 3] -= poses_my_aboba[:, :3, 3].mean(axis=0)

fig1 = plot_poses(poses_my_aboba, k=0.3, text=poses_my_names, fontsize=8, marker_color="red")
fig2 = plot_poses(poses_colmap_aboba, k=0.3, text=poses_colmap_names, fontsize=8, marker_color="blue")
fig = go.Figure(data = fig1.data + fig2.data)
fig.show()

In [143]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [294]:
source = o3d.io.read_point_cloud("./my.pcd")
target = o3d.io.read_point_cloud("./colmap.pcd")
threshold = 0.1

trans_init = np.eye(4)
# trans_init[:3, :3] = scipy.spatial.transform.Rotation.random().as_matrix()
# draw_registration_result(source, target, trans_init)
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(source, target,
                                                    threshold, trans_init)
print(evaluation)

print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=500000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

# source_temp = copy.deepcopy(source)
# target_temp = copy.deepcopy(target)
# source_temp.paint_uniform_color([1, 0.706, 0])
# target_temp.paint_uniform_color([0, 0.651, 0.929])
# source_temp.transform(reg_p2p.transformation)

# plot_poses(np.asarray(source_temp.points), k=0.3, text=poses_my_names, fontsize=8, only_points=True)
# plot_poses(np.asarray(target.points), k=0.3, text=poses_colmap_names, fontsize=8, only_points=True)

Initial alignment
RegistrationResult with fitness=5.250000e-01, inlier_rmse=6.378629e-02, and correspondence_set size of 21
Access transformation to get result.
Apply point-to-point ICP
RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.634803e-02, and correspondence_set size of 40
Access transformation to get result.
Transformation is:
[[ 9.96679045e-01  8.08520314e-02  9.68664573e-03  3.46243128e-05]
 [-8.07084612e-02  9.96633804e-01 -1.43946184e-02  1.18409154e-03]
 [-1.08178727e-02  1.35650202e-02  9.99849471e-01 -7.96354290e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
# plot_poses(np.asarray(source_temp.points), k=0.3, text=poses_my_names, fontsize=8, only_points=True)

### Посмотрим на преобразования, которые совершаются

In [ ]:
# pose0 = np.eye(4)[None, ...]
# pose0[:, 0:3, 3] = [1.0, 2.0, 3.0]

pose1 = np.eye(4)[None, ...]
pose1[:, 0:3, 3] = [1.0, 2.0, 3.0]
# flip axes
pose1[:, 0:3, 1] *= -1
pose1[:, 0:3, 2] *= -1

pose2 = np.eye(4)[None, ...]
pose2[:, 0:3, 3] = [1.0, 2.0, 3.0]
# flip axes
pose2[:, 0:3, 1] *= -1
pose2[:, 0:3, 2] *= -1
# swap
pose2 = pose2[:, [1, 0, 2, 3], :] # swap y and z

pose3 = np.eye(4)[None, ...]
pose3[:, 0:3, 3] = [1.0, 2.0, 3.0]
# flip axes
pose3[:, 0:3, 1] *= -1
pose3[:, 0:3, 2] *= -1
# swap
pose3 = pose3[:, [1, 0, 2, 3], :] # swap y and z
pose3[:, 2, :] *= -1 # flip whole world upside down

# poses = np.stack([pose0, pose1, pose2, pose3])[:, 0, :, :]
poses = np.stack([pose1, pose2, pose3])[:, 0, :, :]

# fig = plot_poses(poses, k=0.5, text=["begin", "flip camera", "swap", "flip world"])
fig = plot_poses(poses, k=0.5, text=["flip camera", "swap", "flip world"])
fig.update_layout(
    scene=dict(
        xaxis = dict(range=[-3,3],),
        yaxis = dict(range=[-3,3],),
        zaxis = dict(range=[-3,3],),
    )
)
fig.show()

In [132]:
np.stack([pose0, pose1, pose2])[:, 0, :, :].shape

(3, 4, 4)

In [313]:
xxx = []
xxx.append(np.ones((5, 4)))
xxx.append(np.zeros((5, 4)))
xxx.append(np.ones((5, 4)))
np.vstack(xxx)

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])